In [6]:
# ==========================================
# Cell 1: Load requirements.json (with stakeholders)
# ==========================================
import json
import pathlib
import pandas as pd

def find_repo_root(start=pathlib.Path.cwd()):
    for cand in [start] + list(start.parents):
        if (cand / 'build.gradle').exists() or (cand / '.git').exists():
            return cand
    return pathlib.Path.cwd()

repo_root = find_repo_root()
req_path = repo_root / 'build' / 'results' / 'mission-control' / 'requirements.json'
if not req_path.exists():
    raise FileNotFoundError(f"Requirements file not found at {req_path} (repo_root={repo_root})")

with req_path.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])
rows = []

def short_from_uri(u):
    if not u:
        return ''
    if '#' in u:
        return u.split('#')[-1]
    return u.rstrip('/').split('/')[-1]

for b in bindings:
    req_uri = b.get('requirementURI', {}).get('value')
    desc = b.get('description', {}).get('value')
    expr = b.get('expression', {}).get('value')
    stakeholder_uri = b.get('stakeholderURI', {}).get('value')

    name = short_from_uri(req_uri)
    stakeholder = short_from_uri(stakeholder_uri)

    rows.append({
        'Name': name,
        'Description': desc,
        'Expression': expr,
        'Stakeholder': stakeholder
    })

df = pd.DataFrame(rows)
df

,Name,Description,Expression,Stakeholder
0,R3,AI Warden Interface,"The system must enable direct, smooth communic...",Operator
1,R7,Decision Approval Mechanism,The system must allow human approval for AI Fi...,CommandCenter
2,R2,Critical Alerts,The system must show real-time critical alerts...,SafetyOfficer
3,R6,Interoperability with Fire Cloud,The system must ingest and interpret data stre...,SystemEngineer
4,R1,Real-Time Map,The system must display real-time fire and dro...,Operator
5,R5,Historical Views,The system must support views of historical fi...,Operator
6,R4,Adaptive User Experience,The system must remain functional under field ...,FireFighter


In [7]:
# ==========================================
# Cell 2: Build mission-capability-requirement graph
# ==========================================
import networkx as nx
import plotly.graph_objects as go

cap_path = repo_root / 'build' / 'results' / 'mission-control' / 'missions_capabilities.json'
if not cap_path.exists():
    raise FileNotFoundError(f"Capabilities file not found at {cap_path} (repo_root={repo_root})")

with cap_path.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])

nodes = {}
edges = set()

for b in bindings:
    req_uri = b.get('requirementURI', {}).get('value')
    req_desc = b.get('requirementDescription', {}).get('value')
    cap_uri = b.get('capabilityURI', {}).get('value')
    cap_desc = b.get('capabilityDescription', {}).get('value')
    sub_uri = b.get('subCapabilityURI', {}).get('value')
    sub_desc = b.get('subCapabilityDescription', {}).get('value')

    req_name = short_from_uri(req_uri)
    cap_name = short_from_uri(cap_uri)
    sub_name = short_from_uri(sub_uri) if sub_uri else None

    if req_name:
        if req_name not in nodes:
            nodes[req_name] = req_desc
    if cap_name:
        if cap_name not in nodes:
            nodes[cap_name] = cap_desc
    if sub_name:
        if sub_name not in nodes:
            nodes[sub_name] = sub_desc

    # requirement → capability
    if req_name and cap_name:
        edges.add((req_name, cap_name))
    # capability → subcapability
    if cap_name and sub_name:
        edges.add((cap_name, sub_name))

G = nx.Graph()
for n, d in nodes.items():
    G.add_node(n, description=d)
for a, b in edges:
    G.add_edge(a, b)

pos = nx.spring_layout(G, seed=42)

# --- Build Plotly traces ---
edge_x, edge_y = [], []
for u, v in G.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y, mode='lines',
    line=dict(width=1, color='#888'),
    hoverinfo='none'
)

node_x, node_y, hover_text, labels = [], [], [], []
for n in G.nodes():
    x, y = pos[n]
    node_x.append(x)
    node_y.append(y)
    desc = G.nodes[n].get('description') or ''
    hover_text.append(f"{n}<br>{desc}")
    labels.append(n)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=labels,
    textposition='top center',
    hovertext=hover_text,
    hoverinfo='text',
    marker=dict(size=18, color='LightSkyBlue', line=dict(width=1, color='DarkSlateGrey'))
)

fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title='Requirement–Capability–SubCapability Graph',
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40)
    )
)

fig.show()


In [9]:
# ==========================================
# Cell 3: Capability → Entity Graph
# ==========================================
ent_path = repo_root / 'build' / 'results' / 'mission-control' / 'entities.json'
if not ent_path.exists():
    raise FileNotFoundError(f"Entities file not found at {ent_path} (repo_root={repo_root})")

with ent_path.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])
nodes = {}
edges = set()

for b in bindings:
    cap_uri = b.get('capabilityURI', {}).get('value')
    cap_desc = b.get('capabilityDescription', {}).get('value')
    ent_uri = b.get('entityURI', {}).get('value')
    ent_desc = b.get('entityDescription', {}).get('value')

    cap_name = short_from_uri(cap_uri)
    ent_name = short_from_uri(ent_uri)

    if cap_name:
        if cap_name not in nodes:
            nodes[cap_name] = cap_desc
    if ent_name:
        if ent_name not in nodes:
            nodes[ent_name] = ent_desc
    if cap_name and ent_name:
        edges.add((cap_name, ent_name))

G_entities = nx.Graph()
for n, d in nodes.items():
    G_entities.add_node(n, description=d)
for a, b in edges:
    G_entities.add_edge(a, b)

pos = nx.spring_layout(G_entities, seed=42)

# --- Build edge trace ---
edge_x, edge_y = [], []
for u, v in G_entities.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y, mode='lines',
    line=dict(width=1, color='#888'),
    hoverinfo='none'
)

# --- Build node trace ---
node_x, node_y, hover_text, labels = [], [], [], []
for n in G_entities.nodes():
    x, y = pos[n]
    node_x.append(x)
    node_y.append(y)
    desc = G_entities.nodes[n].get('description') or ''
    hover_text.append(f"{n}<br>{desc}")
    labels.append(n)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=labels,
    textposition='top center',
    hovertext=hover_text,
    hoverinfo='text',
    marker=dict(size=18, color='LightGreen', line=dict(width=1, color='DarkSlateGrey'))
)

fig_entities = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title='Capability–Entity Graph',
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40)
    )
)

fig_entities.show()


In [10]:
# ==========================================
# Cell 5: Export all visuals into one HTML dashboard
# ==========================================
from pathlib import Path

table_html = df.to_html(classes='table table-striped table-bordered', index=False)
fig_missions_html = fig.to_html(full_html=False, include_plotlyjs=False)
fig_entities_html = fig_entities.to_html(full_html=False, include_plotlyjs=False)

html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Mission Control Dashboard</title>
    <link rel="stylesheet"
          href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link rel="stylesheet"
          href="https://cdn.datatables.net/1.13.6/css/dataTables.bootstrap5.min.css">
    <script src="https://cdn.jsdelivr.net/npm/jquery@3.7.1/dist/jquery.min.js"></script>
    <script src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>
    <script src="https://cdn.datatables.net/1.13.6/js/dataTables.bootstrap5.min.js"></script>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
      body {{ margin: 2rem; }}
      h1 {{ margin-bottom: 1.5rem; }}
    </style>
</head>
<body>
    <h1>Requirements Table</h1>
    <div class="table-responsive">
      {table_html}
    </div>

    <h1 style="margin-top:3rem;">Requirement–Capability–SubCapability Graph</h1>
    {fig_missions_html}

    <h1 style="margin-top:3rem;">Capability–Entity Graph</h1>
    {fig_entities_html}

    <script>
    $(document).ready(function() {{
        $('table').DataTable({{
            paging: true,
            searching: true,
            ordering: true,
            pageLength: 10
        }});
    }});
    </script>
</body>
</html>
"""

out_path = Path("mission_dashboard.html")
out_path.write_text(html, encoding="utf-8")
print(f"✅ Dashboard saved to {out_path.resolve()}")

✅ Dashboard saved to /Users/vivek/Notes/CS_188/mission-control/notebooks/mission_dashboard.html
